In [13]:
import requests
import numpy as np
import pandas as pd
import math

import json
import time
from datetime import datetime, timedelta
import sgp4
from sgp4.api import Satrec, WGS72
from sgp4.conveniences import jday
from sgp4.ext import invjday

import matplotlib.pyplot as plt

In [14]:
username = 'asas4539@hanyang.ac.kr'
password = 'onsaemiro1729!!'

In [15]:
def get_tle():
    # Space-Track API의 URL
    url = "https://www.space-track.org/ajaxauth/login"

    # 요청에 필요한 인증 정보
    payload = {"identity": username, "password": password}

    # Session 객체 생성
    session = requests.Session()

    # 로그인 요청
    response = session.post(url, data=payload)

    if response.status_code != 200:
        print("로그인에 실패했습니다.")
        return None

    # TLE 요청
    response = session.get(
        f"https://www.space-track.org/basicspacedata/query/class/tle/NORAD_CAT_ID/58400/EPOCH/2022-12-31--2024-07-25/orderby/EPOCH%20asc/format/3le/emptyresult/show"
    )#

    return response

In [16]:
def get_info(seg):
    change_df_seg = seg

    start_info = change_df_seg.iloc[0]
    epoch = datetime(start_info['year'], start_info['month'], start_info['day'], start_info['hour'], start_info['minute'], int(start_info['second']))
    epoch += timedelta(seconds=1)
    
    end_info = change_df_seg.iloc[-1]
    end_epoch = datetime(end_info['year'], end_info['month'], end_info['day'], end_info['hour'], end_info['minute'], int(end_info['second']))

    all_info = []

    earth_radius = 6378.137
    rad2deg = 180.0 / 3.141592653589793

    while epoch <= end_epoch:
        # print(epoch)
        
        year = epoch.year
        month = epoch.month
        date = epoch.day
        hour = epoch.hour
        minute = epoch.minute
        second = epoch.second

        now_time = str(year) +'-' + str(month) + '-' + str(date) + ' ' + str(hour) + ':' + str(minute) + ':' + str(second)
        temp = change_df_seg[change_df_seg['time'] <= pd.to_datetime(now_time)].copy()

        jd, fr =jday(year, month, date, hour, minute, second)

        tle1, tle2 = temp.iloc[-1]['first_line'], temp.iloc[-1]['second_line']
        satellite = Satrec.twoline2rv(tle1, tle2)

        e, position, velocity = satellite.sgp4(jd, fr)

        all_info.append([len(temp), tle1, tle2,
                        year, month, date, hour, minute, second, 
                        position[0], position[1], position[2], velocity[0], velocity[1], velocity[2],
                        (position[0]**2 + position[1]**2 + position[2]**2)**0.5, (velocity[0]**2 + velocity[1]**2 + velocity[2]**2)**0.5,
                        satellite.alta * earth_radius , satellite.altp * earth_radius,
                        satellite.inclo*rad2deg, satellite.ecco, satellite.nodeo])

        epoch += timedelta(seconds=1)

    info_df = pd.DataFrame(all_info)
    info_df.columns = ['change', 'tle1','tle2',
                        'year', 'month', 'day', 'hour', 'minute', 'second',
                    'x', 'y', 'z', 'vx', 'vy', 'vz',
                        'altitude', 'velocity', 
                        'apogee', 'perigee', 
                        'inclination', 'eccentricity', 'raan']
        
    return info_df

In [17]:
response = get_tle()
tle_datas = response.text.split('\r\n')

tle_first_lst = []
tle_second_lst = []

for i in range(0, len(tle_datas)-2, 3):
    tle_first_lst.append(tle_datas[i+1])
    tle_second_lst.append(tle_datas[i+2])

tle_df = pd.DataFrame({'first_line':tle_first_lst, 'second_line':tle_second_lst})
tle_df.drop_duplicates(keep='first', inplace=True)
tle_df.reset_index(inplace=True, drop=True)

In [18]:
change_times = []
for i in range(len(tle_df)):
    tle_1, tle_2 = tle_df['first_line'].iloc[i], tle_df['first_line'].iloc[i]
    satellite = Satrec.twoline2rv(tle_1,tle_2)
    jdsatepoch = satellite.jdsatepoch
    jdsatepochfrac = satellite.jdsatepochF
    epochdatetime = invjday(jdsatepoch + jdsatepochfrac)
    change_times.append(epochdatetime)
    
change_df = pd.DataFrame(change_times)
change_df.columns = ['year', 'month', 'day', 'hour', 'minute', 'second']
change_df['first_line']=tle_df['first_line']
change_df['second_line']=tle_df['second_line']
change_df['time'] = change_df['year'].astype('str') + '-' + change_df['month'].astype('str') + '-' + change_df['day'].astype('str') + ' ' + change_df['hour'].astype('str') + ':' + change_df['minute'].astype('str') + ':' + change_df['second'].astype('int').astype('str')
change_df['time'] = pd.to_datetime(change_df['time'])

In [19]:
end_point = change_df.iloc[-1:].copy()

end_point['day'] = end_point['day'] + 1
end_point['hour'] = 0
end_point['minute'] = 0
end_point['second'] = 0

end_point['time'] = pd.to_datetime(end_point['year'].astype('str') + '-' + end_point['month'].astype('str') + '-' + end_point['day'].astype('str') + ' ' + end_point['hour'].astype('str') + ':' + end_point['minute'].astype('str') + ':' + end_point['second'].astype('int').astype('str'))
end_point

,year,month,day,hour,minute,second,first_line,second_line,time
1098,2024,7,25,0,0,0,1 58400U 23179A 24206.93643670 .00007378 0...,2 58400 97.3982 93.1451 0003545 80.4808 279...,2024-07-25


In [20]:
change_df = pd.concat([change_df,end_point], axis=0).copy()
change_df.reset_index(inplace=True, drop=True)

In [21]:
day_change_idx = list(change_df[['year','month','day']].drop_duplicates().index)

In [22]:
seg_lst = []

for i in range(len(day_change_idx)-1):
    seg_lst.append(change_df.iloc[day_change_idx[i]:day_change_idx[i+1]+1].copy())

In [23]:
from joblib import Parallel, delayed
from tqdm import tqdm

In [24]:
results = Parallel(n_jobs=5)(delayed(get_info)(seg) for seg in tqdm(seg_lst))


  0%|          | 0/246 [00:00<?, ?it/s]
KeyboardInterrupt



In [13]:
rslt = pd.concat(results)

In [16]:
rslt.sort_values(by=['year', 'month', 'day', 'hour', 'minute', 'second']).reset_index(drop=True).to_csv('58400_persec.csv')

In [ ]:
#info_df.to_csv('39227.csv', index=False)

In [ ]:
# info_df.to_csv('25544.csv', index=False)

In [ ]:
# info_df.to_csv('51130.csv', index=False)